# *Library*
Import *library* yang dibutuhkan.

In [ ]:
!pip3 install num2words # Install library untuk mengubah angka menjadi kata.
!pip install Sastrawi # Karena wordnet tidak bisa digunakan maka di sini pakai Sastrawi.

In [ ]:
# Library untuk membaca file .csv.
import pandas as pd
# Library untuk visualisasi.
import matplotlib.pyplot as plt
import seaborn as sns
# Library untuk memproses teks.
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.text import one_hot
from num2words import num2words
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# Library untuk membangun model.
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
import numpy as np
# Library untuk memisahkan dataset.
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# Library untuk evaluasi model.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
nltk.download('punkt') # Untuk tokenizer kata.
nltk.download('stopwords')# Untuk menghapus stopwords.
# nltk.download('wordnet') # Ada masalah ketika menggunakan wordnet.

# Membaca Dataset
Melakukan visualisasi dataset yang dimiliki.

In [ ]:
df = pd.read_csv("/kaggle/input/data-mh-id/data_mh.csv", delimiter=",") # Sesuaikan `delimiter` berdasarkan bahasa yang digunakan dalma .csv.
df.head()

Visualisasi label, usahakan label yang dimiliki seimbang.

In [ ]:
df["is_depression"].value_counts().plot(kind="bar",figsize=(5,3))

In [ ]:
# Cek apakah terdapat null value atau tidak.
print(df.isnull().sum())

In [ ]:
# Cek tipe data kolom.
print(df.info())

# Memproses Data Teks
Proses data teks seperti menghapus `stop_words`, menjadikannya kata dasar, mengganti angka ke dalam teks, dll.

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

# def translate_text(text):
#     url = "https://translate.googleapis.com/translate_a/single?client=gtx&sl=en&tl=id&dt=t&q={}".format(text)
#     response = requests.get(url)
#     result = response.json()[0][0][0]
#     return result

def preprocess_text(text):
#     text = translate_text(text) # Kemudian di sini kita coba translate
    text = word_tokenize(text.lower()) # Tokenize text_id ke dalam token/kata
    text = [t for t in text if t not in stop_words] # Hapus stop_words
    text = [stemmer.stem(t) for t in text] # Menjadikan kata dasar
    text = [t if not t.isdigit() else num2words(int(t)) for t in text] # Mengganti angka ke teks
#     text = emoji.demojize(text) # ubah emoji ke dalam bentuk teks
#     text = re.sub(r':[a-z_]+:', lambda m: ' '.join(m.group(0).replace(':', '').split('_')), text) # Regex misal emoji: 🎉 bakal diubah jadi "party popper".
    text = ' '.join(text) # Gabungkan ke dalam teks kembali
    return text

In [ ]:
df = df[pd.notnull(df['text_id'])] # hapus semua baris dengan nilai NaN di kolom `text`
df = df[df['text_id'].apply(lambda x: isinstance(x, str))] # hapus semua baris di mana `text` bukan string
df['nlp_text'] = df['text_id'].apply(lambda x: preprocess_text(x))

In [ ]:
df.head()

# Hitung Ukuran *Vocabulary*
Memasukan kata unik ke dalam *se* `words`, kemudian untuk menghitung ukuran *vocabulary*.

In [ ]:
words = set(word for sentence in df["nlp_text"] for word in sentence.split())
vocab_size = len(words)
print(vocab_size)

# *One Hot Encoding* untuk Setiap Kalimat
*One-hot encode* teks ke dalam daftar indeks kata berukuran n.

In [ ]:
one_hot_representation = [one_hot(words,vocab_size) for words in df["nlp_text"]] # Input teksnya `words` di setiap `df["nlp_text"]` dengan ukuran `vocab_size`
# one_hot_representation

Cari panjang maksimum untuk dimasukan ke dalam word embeddings.

In [ ]:
max_len = 0
for elem in one_hot_representation:
    if len(elem) > max_len:
        max_len = len(elem)
max_len

# Mengonversi *One-hot Encodings* ke *Word Embeddings*
Meskipun *one-hot encodings* adalah teknik yang berguna untuk merepresentasikan kata-kata sebagai vektor numerik di NLP, teknik ini memiliki beberapa keterbatasan. Salah satunya adalah vektor *one-hot* berdimensi tinggi dan *sparse*, yang berarti bahwa sebagian besar elemen vektor adalah 0, yang dapat mempersulit algoritma pembelajaran mesin untuk mengekstraksi pola yang bermakna dari data.
Untuk mengatasi keterbatasan ini, kita dapat mengonversi vektor *one-hot* menjadi *word embeddings*, yaitu vektor yang sifatnya *dense* berdimensi rendah yang merepresentasikan kata dalam ruang vektor kontinu.

Ketika panjang sequences kurang dari max_length maka pad_sequences akan menambahkan padding supaya jumlahnya sama dengan max_length. padding = 'post' dibuat untuk menambahkan padding di akhir, jika padding = 'pre' maka padding akan ditambahkan di awal sequences.

Kemudian apabila panjang sequences lebih dari max_length maka akan ditruncate. padding = 'post' dibuat untuk truncate di akhir, jika padding = 'pre' maka dibuat untuk truncate di awal.

In [ ]:
# embedded_docs = pad_sequences(one_hot_representation, padding='pre', maxlen = max_len)
embedded_docs = pad_sequences(one_hot_representation, padding = 'post',truncating = 'post', maxlen = max_len)

In [ ]:
### Model 1
# embedding_vector_features = max_len * 2
# model = Sequential()
# model.add(Embedding(14233, embedding_vector_features, input_length=max_len))
# model.add((LSTM(100)))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# print(model.summary())

In [ ]:
### Model 2
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=14233, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
y = df["is_depression"]

Jumlah sample `y` dengan `embedded_docs` harus sama.

In [ ]:
print(f"Shape of y is: {y.shape}")
print(f"Shape of embedded document is: {embedded_docs.shape}")

# Train Test Split

Di sini kita split `test_size` sebesar 0.1 dikarenakan jumlah datasetnya berukuran besar. Kemudian menggunakan `stratifikasi` dalam fungsi `train_test_split()` memastikan bahwa `training set` dan `test set` memiliki distribusi sampel yang proporsional dari setiap label kelas dalam variabel target, yang dapat membantu jika set data tidak seimbang.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size = 0.1, random_state = 42, stratify = y)

In [ ]:
print(f"Shape of X train is: {X_train.shape}")
print(f"Shape of y train is: {y_train.shape}")
print(f"Shape of X test is: {X_test.shape}")
print(f"Shape of y test is: {y_test.shape}")

Pastikan *X train* dengan *y train* punya jumlah sampel yang sama dan *X test* dengan *y test* punya jumlah sampel yang sama.

Selain itu pastikan kolom *X train* dengan *X test* berjumlah sama dan *y train* dengan *y test* berjumlah sama.

**Contoh: (jumlah sampel, jumlah kolom)**

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=3,batch_size=16) # Kayanya epoch cukup di 3 aja

#### Kalau ga cukup coba pakai chunking
# chunk_size = 1000  # set the size of each chunk
# # shuffle the data
# X_train, Y_train = shuffle(X_train, Y_train)
# # loop through the data in chunks
# for i in range(0, len(X_train), chunk_size):
#     X_chunk = X_train[i:i+chunk_size]
#     Y_chunk = Y_train[i:i+chunk_size]  
#     # train the model on the current chunk
#     model.fit(X_chunk, Y_chunk, epochs=1, batch_size=32, validation_data=(X_test, Y_test))

In [ ]:
y_pred = model.predict(X_test)

Probabilitas lebih besar dari atau sama dengan 0,5 akan diberikan ke kelas positif.

In [ ]:
y_pred = (y_pred >= 0.5).astype("int")

In [ ]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [ ]:
y_test = y_test.reset_index(drop = True).values # Reset index `y_test` supaya dari 0 lagi sebelum dikonversi ke dalam array

In [ ]:
# model.save('model.h5')

In [ ]:
#### Belum selesai (sebentar aku beresin)
# from keras.models import load_model

# model = load_model('model.h5')

# # Take user input from the command line
# text = input("Enter your text: ")

# # Tokenize and preprocess the text (use the same preprocessing steps as in training)
# preprocessed_text = preprocess(text)

# # Convert preprocessed text to a numpy array
# X = np.array([preprocessed_text])

# # Make predictions using your trained model
# y_pred_prob = model.predict(X)
# y_pred = np.argmax(y_pred_prob, axis=1)

# # Print the predicted label (0 or 1)
# print("Predicted label: {}".format(y_pred[0]))

In [ ]:
for i in range(len(y_test)):
    print("True label: {}, Predicted label: {}".format(y_test[i], y_pred[i]))

In [ ]:
h = history.history
print(h.keys())

In [ ]:
# Plot fungsi loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Plot akurasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()